In [50]:
import numpy as np
import keras
import cv2
import pandas as pd
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import InputLayer
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
import itertools
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [51]:
file_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\img'
labels_path = 'D:\Admin\Documents\Year_4\AMLS\Assessment\dataset_AMLS_20-21\celeba\labels.csv'

In [100]:
dft = pd.read_csv(labels_path, skiprows=lambda x: x in range(1,5000,5))
dft["smiling"] = dft["smiling"].replace(to_replace=[-1], value=['Frown'])
dft["smiling"] = dft["smiling"].replace(to_replace=[1], value=['Smile'])
dft["gender"] = dft["gender"].replace(to_replace=[-1], value=['Female'])
dft["gender"] = dft["gender"].replace(to_replace=[1], value=['Male'])
one_hot_s = pd.get_dummies(dft["smiling"])
one_hot_g = pd.get_dummies(dft["gender"])
dft = dft.drop(columns=["gender", "smiling"])
dft = dft.join(one_hot_g)
dft = dft.join(one_hot_s)
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_dataframe(
    dataframe=dft, 
    directory=file_path,
    x_col="img_name", 
    y_col=["Frown", "Smile"], 
    class_mode="raw", 
    target_size=(55,45), 
    batch_size=100, 
    color_mode='grayscale', 
    interpolation='bicubic')

Found 4000 validated image filenames.


In [96]:
df

,Unnamed: 0,img_name,Female,Male,Frown,Smile
0,1,1.jpg,1,0,0,1
1,2,2.jpg,0,1,1,0
2,3,3.jpg,1,0,1,0
3,4,4.jpg,1,0,1,0
4,6,6.jpg,0,1,1,0
...,...,...,...,...,...,...
3995,4994,4994.jpg,0,1,0,1
3996,4996,4996.jpg,0,1,0,1
3997,4997,4997.jpg,0,1,0,1
3998,4998,4998.jpg,0,1,0,1


In [102]:
dfv = pd.read_csv(labels_path, skiprows=lambda x: x not in range(0,5000,5))
dfv["smiling"] = dfv["smiling"].replace(to_replace=[-1], value=['Frown'])
dfv["smiling"] = dfv["smiling"].replace(to_replace=[1], value=['Smile'])
dfv["gender"] = dfv["gender"].replace(to_replace=[-1], value=['Female'])
dfv["gender"] = dfv["gender"].replace(to_replace=[1], value=['Male'])
one_hot_s = pd.get_dummies(dfv["smiling"])
one_hot_g = pd.get_dummies(dfv["gender"])
dfv = dfv.drop(columns=["gender", "smiling"])
dfv = dfv.join(one_hot_g)
dfv = dfv.join(one_hot_s)
datagen = ImageDataGenerator(rescale=1./255)
test_generator = datagen.flow_from_dataframe(
    dataframe=dfv, 
    directory=file_path,
    x_col="img_name", 
    y_col=["Frown", "Smile"], 
    class_mode="raw", 
    target_size=(55,45), 
    batch_size=100, 
    color_mode='grayscale', 
    interpolation='bicubic')

Found 999 validated image filenames.


In [103]:
model = Sequential()
model.add(InputLayer(input_shape=(55,45,1)))
model.add(Conv2D(filters=100, kernel_size=(2,2), activation='relu', strides=2))
model.add(Dropout(0.25))
model.add(Conv2D(filters=100, kernel_size=(2,2), activation='relu', strides=2))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(256))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [104]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 27, 22, 100)       500       
_________________________________________________________________
dropout_29 (Dropout)         (None, 27, 22, 100)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 13, 11, 100)       40100     
_________________________________________________________________
dropout_30 (Dropout)         (None, 13, 11, 100)       0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 5, 100)         0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 6, 5, 100)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 3000)             

In [105]:
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [106]:
train_steps = train_generator.n//train_generator.batch_size
test_steps = test_generator.n//test_generator.batch_size

In [108]:
model.fit_generator(generator=train_generator, 
                    steps_per_epoch=train_steps,
                    validation_data=test_generator,
                    validation_steps=test_steps,
                    epochs=50)

Epoch 1/50
40/40 [==============================] - 9s 216ms/step - loss: 0.6268 - accuracy: 0.6520 - val_loss: 0.5937 - val_accuracy: 0.7222
Epoch 2/50
40/40 [==============================] - 9s 228ms/step - loss: 0.6236 - accuracy: 0.6465 - val_loss: 0.5667 - val_accuracy: 0.7275
Epoch 3/50
40/40 [==============================] - 8s 210ms/step - loss: 0.6048 - accuracy: 0.6718 - val_loss: 0.5422 - val_accuracy: 0.7419
Epoch 4/50
40/40 [==============================] - 8s 208ms/step - loss: 0.5996 - accuracy: 0.6845 - val_loss: 0.5516 - val_accuracy: 0.7364
Epoch 5/50
40/40 [==============================] - 9s 213ms/step - loss: 0.5802 - accuracy: 0.6975 - val_loss: 0.5378 - val_accuracy: 0.7475
Epoch 6/50
40/40 [==============================] - 8s 209ms/step - loss: 0.5744 - accuracy: 0.6995 - val_loss: 0.5025 - val_accuracy: 0.7486
Epoch 7/50
40/40 [==============================] - 9s 223ms/step - loss: 0.5628 - accuracy: 0.7155 - val_loss: 0.4633 - val_accuracy: 0.7642
Epoch 